<a href="https://colab.research.google.com/github/Vey27/Reinforcement-Learning/blob/main/LunarLanderSSB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!pip install "gymnasium==0.28.1"

In [29]:
!pip install "gymnasium[atari,accept-rom-license]"

  Using cached AutoROM-0.4.2-py3-none-any.whl (16 kB)
  Attempting uninstall: autorom
    Found existing installation: AutoROM 0.6.1
    Uninstalling AutoROM-0.6.1:
      Successfully uninstalled AutoROM-0.6.1


In [30]:
!pip install gymnasium[classic-control]

In [31]:
!pip install gymnasium[other]

In [32]:
!pip install --upgrade pip


In [33]:
!apt-get install -y swig


Reading package lists... Done
Building dependency tree       
Reading state information... Done
swig is already the newest version (4.0.1-5build1).
0 upgraded, 0 newly installed, 0 to remove and 13 not upgraded.


In [34]:
!pip install gymnasium[box2d]

In [35]:
!pip install stable-baselines3

In [36]:

!pip install stable-baselines3[extra]


  Using cached AutoROM-0.6.1-py3-none-any.whl (9.4 kB)
  Attempting uninstall: autorom
    Found existing installation: AutoROM 0.4.2
    Uninstalling AutoROM-0.4.2:
      Successfully uninstalled AutoROM-0.4.2


In [37]:
!pip3 install ipywidgets

In [38]:
import gymnasium as gym

from stable_baselines3 import DQN
from stable_baselines3.common.evaluation import evaluate_policy


# Create environment
env = gym.make("LunarLander-v2", render_mode="rgb_array")



In [39]:
# Instantiate the agent
model = DQN("MlpPolicy", env, verbose=1)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [40]:
# Train the agent and display a progress bar
model.learn(total_timesteps=int(2e5), progress_bar=True)


Output()

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 93       |
|    ep_rew_mean      | -245     |
|    exploration_rate | 0.982    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 1328     |
|    time_elapsed     | 0        |
|    total_timesteps  | 372      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 84.9     |
|    ep_rew_mean      | -173     |
|    exploration_rate | 0.968    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 1151     |
|    time_elapsed     | 0        |
|    total_timesteps  | 679      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 79       |
|    ep_rew_mean      | -154     |
|    exploration_rate | 0.955    |
| time/               |          |
|    episodes       

In [41]:
# Save the agent
model.save("dqn_lunar")
#del model  # delete trained model to demonstrate loading



In [42]:
# Load the trained agent
# NOTE: if you have loading issue, you can pass `print_system_info=True`
# to compare the system on which the model was trained vs the current one
# model = DQN.load("dqn_lunar", env=env, print_system_info=True)
model = DQN.load("dqn_lunar", env=env)



Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [43]:
# Evaluate the agent
# NOTE: If you use wrappers with your environment that modify rewards,
#       this will be reflected here. To evaluate with original rewards,
#       wrap environment in a "Monitor" wrapper before other wrappers.
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)



In [58]:
import cv2
import base64
from IPython.display import HTML
from google.colab import files

vec_env = model.get_env()
obs = vec_env.reset()
image_list = []  # List to store the rendered images

# Define the desired width for the video
video_width = 600

for i in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, info = vec_env.step(action)
    image = vec_env.render("rgb_array")  # Render the environment as an image

    # Resize the image to fit the desired width
    height, width, _ = image.shape
    aspect_ratio = video_width / float(width)
    resized_image = cv2.resize(image, (video_width, int(height * aspect_ratio)))

    image_list.append(resized_image)  # Add the resized image to the list

# Create an HTML animation
animation_html = """
<video width="{0}" controls>
  <source src="data:video/mp4;base64,{1}" type="video/mp4">
</video>
"""

# Convert the image list to a base64-encoded video
video_data = cv2.VideoWriter("video.mp4", cv2.VideoWriter_fourcc(*"mp4v"), 30, image_list[0].shape[:2][::-1])
for image in image_list:
    video_data.write(image)
video_data.release()

# Download the video file
files.download("video.mp4")

# Read the video file as bytes
video_bytes = open("video.mp4", "rb").read()
# Encode the video bytes as base64
video_base64 = base64.b64encode(video_bytes).decode("utf-8").replace("\n", "")

# Display the HTML animation
HTML(animation_html.format(video_width, video_base64))


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [53]:
env.close()

In [54]:
model.env.close()